In [1]:
!pip install -q faster-whisper ffmpeg-python
!apt-get install -y ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 129.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [5]:
from faster_whisper import WhisperModel
import os

AUDIO_FILE = "/content/caption generation.mp4"

# Extract base filename (without extension)
base_name = os.path.splitext(os.path.basename(AUDIO_FILE))[0]

txt_path = f"/content/{base_name}.txt"
srt_path = f"/content/{base_name}.srt"

# Helper: CapCut-compatible SRT timestamp
def format_srt_time(seconds: float) -> str:
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{secs:02},{millis:03}"

# Load model
model = WhisperModel(
    "large-v3",
    device="cuda",
    compute_type="float16"
)

# Transcribe
segments, info = model.transcribe(
    AUDIO_FILE,
    beam_size=5,
    language='en',
    condition_on_previous_text=False,
    vad_filter=True
)

print(f"Detected language: {info.language}")
print(f"Output files: {base_name}.txt, {base_name}.srt")

# Write outputs
with open(txt_path, "w", encoding="utf-8") as txt, \
     open(srt_path, "w", encoding="utf-8") as srt:

    for i, segment in enumerate(segments, start=1):
        text = segment.text.strip()

        txt.write(text + "\n")

        start = format_srt_time(segment.start)
        end = format_srt_time(segment.end)

        srt.write(f"{i}\n")
        srt.write(f"{start} --> {end}\n")
        srt.write(text + "\n\n")

print("✅ Transcription complete")


Detected language: en
Output files: caption generation.txt, caption generation.srt
✅ Transcription complete
